atomic type theory
Named nones
What about a model where everything has None in it?
The "var" abstraction. Could be seen as an abstract interpretation. Constant propagation. None = dunno

Dependent egraph ignoring hash cons becomes dependent 

Lambda free dependent types is roughly GATs? Which is roughly essentialy algebraic theories?
Which is roughly 


In [ ]:
# just get rid of objects
Arr = smt.DeclareSort("Arr")
dom = smt.Function("dom", Arr, Arr) # The identity arrow at the dom stands for
cod = smt.Function("cod", Arr, Arr)
comp = smt.Function("comp", Arr, Arr, Arr)
# id typing
dom(dom(f)) == dom(f)
cod(dom(f)) == dom(f)
dom(cod(f)) == cod(f)
cod(cod(f)) == cod(f)
comp_dom = comp(f, dom(f)) == f
cod_comp = comp(cod(f), f) == f
comp_assoc = smt.ForAll([f,g,h], cod(g) == dom(f), dom(h) == cod(g)], comp(comp(g, f), h) == comp(g, comp(f, h)))

C = smt.SetSort(Arr)

is_cat = kd.define("is_cat", [C],
    smt.ForAll([f], C[f], C[dom(f)]),
    smt.ForAll([f], C[f], C[cod(f)]),
    smt.ForAll([f,g], C[f], C[g], cod(g) == dom(f), C[comp(g, f)])
)

F = smt.Array("F", Arr, Arr)
is_functor = kd.define("is_functor", [F, C, D],
    smt.ForAll([f], C[f], D[F[f]]),
    smt.ForAll([f], C[f], F[dom(f)] == dom(F[f)]),
    smt.ForAll([f], C[f], F[cod(f)] == cod(F[f)]),
    smt.ForAll([f,g], C[f], C[g], cod(g) == dom(f), F[comp(g, f)] == comp(F[g], F[f]))
)

functor_compose = kd.prove(smt.ForAll([F,G,C,D,E], is_functor(F, C,D), is_functor(G,D,E),
    is_functor(smt.Lambda([f],G[F[f]]), C, E)
))
is_obj = kd.define("is_id" , [f], f == dom(f), f == cod(f))
hom = kd.define("hom", [C, A, B])

term = kd.define("term", [f,C],
    is_ob(f),

    smt.ForAll([g])
)














ambient wf is kind of nice.



In [ ]:
Category = kd.Struct("Category",
    ("hom", Ob, Ob, smt.SetSort(Arr)),
    ("id", smt.ArraySort(Ob, smt.BoolSort()),
    ("comp", smt.ArraySort(Arr, Arr, Arr))
))
hom = smt.Const("hom", smt.FunctionSort(Ob, Ob, smt.SetSort(Arr)))

In [ ]:
from kdrag.all import *
Arr = smt.DeclareSort("Arr")

id = smt.Function("id", Arr, Arr)
comp = smt.Function("comp", Arr, Arr, Arr)
kd.notation.matmul.register(Arr, comp)
cod = smt.Function("cod", Arr, Ob)
dom = smt.Function("dom", Arr, Ob)

C = smt.Array("C", Ob, Ob, smt.SetSort(Arr))
D = smt.Array("D", Ob, Ob, smt.SetSort(Arr))
C = smt.SetSort(Arr)
D = smt.SetSort(Arr)

is_cat = kd.define("is_cat", [C],
                   smt.And(
                    smt.ForAll([f], C[f], C[id(dom(f))], C[id(cod(f))]),   
                    smt.ForAll([f,g], C(dom(f),cod(f))[f], C(dom(g), cod(g))[g], C(dom(comp(f,g)), cod(comp(f,g)))[comp(f,g)])
                   )
                   # plays nice with id and comp
                   )


#termarrow = smt.Function("tarr", )
# terminal object
term = kd.define("term",[C,a], 
    smt.ForAll([b], smt.Exists([f], C[f], hom(b,a)[f]))
    smt.ForAll([g,f,b], C[f], hom(b,a)[f], hom(b,a)[g], g == f)
)
F = smt.ArraySort("F", Arr, Arr)
# F is a functor from C to D (it may also be a functor on different domains.
is_functor(  , [F,C,D], 
           smt.And(
               is_cat(C),
               is_cat(D),
           )
           )
# natrual trasnformations
is_nat = kd.define("is_nat", [C,D,F,G],
                   smt.And(
                          is_functor(F, C, D),
                          is_functor(G, C, D),
                   )
                   )









In [ ]:
from kdrag.all import *

Ob = smt.DeclareSort("Ob")
Arr = smt.DeclareSort("Arr")
dom = smt.Function("dom", Arr, Ob)
cod = smt.Function("cod", Arr, Ob)
a,b,c,d = smt.Consts("a b c d", Ob)
f = smt.Const("f", Arr)
#hom = smt.Function("hom", Ob, Ob, smt.SetSort(Arr))
hom = kd.define("hom", [a,b], smt.Lambda([f], smt.And(dom(f) == a, cod(f) == b)))
# hom = kd.define("hom", [a,b,f], smt.And(dom(f) == a, cod(f) == b)) ?
kd.contract(hom, [a,b,f] ,smt.And(dom(f) == a, cod(f) == b)) # Hmm. Fully applied curried contract... hmmm.

#f = smt.Const("f", hom(a,b))
f = smt.Const("f", hom(dom(f)), cod(f)) # true though by unfolding
g = smt.Const("g", hom(b,c))
h = smt.Const("h", hom(c,d))

fi = smt.Const("fi", hom(dom(f), cod(f))) # implicit type vars


id_ = smt.Function("id", Ob, Arr)

comp = smt.Function("comp", Arr, Arr, Arr)
comp_type = kd.axiom(smt.ForAll([a,b,c,f,g], hom(a,c)[f @ g]))
comp_type = kd.axiom(smt.ForAll([f,g], cod(f) == dom(g), hom(dom(f), cod(g))[comp(f,g)]))

# hmm. Not quantified enough over objects...
# since it hom(dom,cod) works. Or unfoled.
#kd.contract(comp, [f,g], smt.And(hom(a,b)[f], hom(b,c)[g]), hom(a,c)[f @ g], by=[comp_type])
kd.contract()

matmul = kd.notation.matmul.define([f,g], comp(f,g))

# Is the implicitness worth it?
kd.axiom(smt.ForAll([a,b,f], f @ id_(a) == f))
kd.axiom(smt.ForAll([a,b,f], id_(b) @ f == f))
kd.axiom(smt.ForAll([a,b,c,d,f,g,h], h @ (g @ f) == (h @ g) @ f))


S = smt.Const("S", smt.SetSort(Arr))
is_cat = kd.define("is_cat", [S], 


F = smt.Arra
is_functor = kd.define("is_functor", [F, A, B],

)





AssertionError: 

Atomic type t


Lambda : The Ultimate Pain in the Ass
Lambda free dependent type theory


Replace the complexity of lambda evaluation with the complexity of definitions.

Make an rpython interpreter

Cody said it's possible to avoid lambda, but incoherent to avoid pi.

Why do I get tripped up on Pi?


app(Vec, n)
app(Fin, n)


head : {n} -> Vec succ n -> int

head(N) : head_type(N)
has_type(head(N), head_type(N))
head_type(N) = vec(succ(N))
head(N)(cons()) = 

pi(forall X, ) ?

head_type(N) : unin(z)
univ(z) : univ(succ(z))


concat_type(N, M) = arr(vec(N), vec(M), vec(N + M))

so no pi?


Inseatd of fres hconstnatns for each new type, we could post a function `typ` tah tdoes it?


In [ ]:
Term = smt.DeclareSort("Term")
Type = smt.DeclareSort("Type")

# typ(head) is the Pi
# positing a functional relationship between terms and types is a choice.
typ_lam : smt.ArraySort(smt.ArraySort(Term, Term), smt.ArraySort(Term, Type))



In [ ]:
from typing import NamedTuple
class Defn(NamedTuple):
    name: str
    args: list[str]
    body: object






In [ ]:
import rply
lg = rply.LexerGenerator()

lg.add("LPAREN", r"\(")
lg.add("RPAREN", r"\)")
lg.add("ATOM", r"[a-zA-Z_][a-zA-Z0-9_]*")
#lg.add("NUMBER", r"[0-9]+")
lg.ignore(r"\s+")
lexer = lg.build()

pg = rply.ParserGenerator(
    # A list of all token names, accepted by the parser.
    ["LPAREN", "RPAREN", "ATOM"],
)
@pg.production("sexp : LPAREN sexp RPAREN")
def sexp_paren(p):
    return p[1:-1]

@pg.production("sexp : ATOM")
def sexp_atom(p):
    return p[0].getstr()

parser = pg.build()

def parse(s):
    tokens = lexer.lex(s)
    return parser.parse(tokens)

parse("(hello (my name) is philip)")


KeyError: 'sexp*'